In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
import random
import copy
import pickle
from os.path import join as oj
sys.path.insert(1, oj(sys.path[0], 'mnist'))

import visualize as viz
import tiling
import agglomerate
import operator
from cd import cd
from tqdm import tqdm
from collections import defaultdict

import torch
import mnist.mnist as dset
from mnist.mnist import pred_ims
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms
import scores as score_funcs
from agglomerate import get_scores

cs_div = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6']

# load the model
model = dset.Net().cuda()
model.load_state_dict(torch.load('mnist/mnist.model'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
method = 'cd'
sweep_dim = 1
layer = 'softmax'
use_abs = False
num_iters = 6
percentile_include = 99
_, train_loader = dset.load_data(train_batch_size=1, test_batch_size=1,
                                    use_cuda=True, data_dir='mnist/data',
                                    shuffle=False)
model.eval()

Net(
  (conv1): Conv2d (1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50)
  (fc2): Linear(in_features=50, out_features=10)
)

# find top examples, worst examples

In [25]:
preds = []
print(len(train_loader))
for i, im in enumerate(train_loader):
    im_torch, label = Variable(im[0].cuda()), im[1].numpy()[0]
    if i % 1000 == 0:
        print(i)
    pred = model(im_torch).data[0, label]
    preds.append(pred)

10000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [37]:
top_idxs = np.argsort(preds)[::-1]
preds[top_idxs[0]]
top_10 = top_idxs[:10]
bot_10 = top_idxs[-10:][::-1]
print('top idxs', top_10)
print('bot idxs', bot_10)

top idxs [4999 9074 9106 9100 6311 9095 6313 9083 6354 6355]
bot idxs [3520 6597 1299 1681 9009 3767 3333  259 1226 6505]


In [4]:
# Generate bad model
np.random.seed(17042018)
modify_prop = 0.25 #0.15
bad_model = copy.deepcopy(model)
bad_model.eval()
for param in bad_model.parameters():
    num_params = np.prod(list(param.data.size()))
    num_modify = modify_prop * num_params
    to_modify = np.random.choice([True, False], tuple(param.data.size()), 
                                 p=(modify_prop, 1-modify_prop))
    cpu_param = param.data.cpu().numpy()
    #permuted_vars = cpu_param[to_modify]
    cpu_param[to_modify] = np.random.permutation(cpu_param[to_modify])
    param.data = torch.cuda.FloatTensor(cpu_param) 



In [5]:
# Compute accuracy of bad model
num_correct, num_total = 0, 0

for i, im in enumerate(train_loader):
    im_torch, im_orig, label = Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0]
    pred = bad_model(im_torch)
    _, pred = pred[0].max(0)
    pred = pred.data[0]
    if pred == label:
        num_correct += 1
    num_total += 1
    if num_total % 10000 == 0:
        print(num_total)

print("Accuracy of bad model: ", num_correct / num_total)

for i, im in enumerate(train_loader):
    im_torch, im_orig, label = Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0]
    pred = model(im_torch)
    _, pred = pred[0].max(0)
    pred = pred.data[0]
    if pred == label:
        num_correct += 1
    num_total += 1
    if num_total % 10000 == 0:
        print(num_total)

print("Accuracy of good model: ", num_correct / num_total)


10000
Accuracy of bad model:  0.7964
20000
Accuracy of good model:  0.8868


In [6]:
good_correct = {}
bad_correct = {}
for i, im in enumerate(train_loader):
    im_torch, im_orig, lab_num = Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0]
    good_max, good_pred = F.softmax(model(im_torch)).max(1)
    good_pred = good_pred.data[0]; good_max = good_max.data[0]
    bad_max, bad_pred = F.softmax(bad_model(im_torch)).max(1)
    bad_pred = bad_pred.data[0]; bad_max = bad_max.data[0]
    
    if (good_pred == lab_num or bad_pred == lab_num) and good_pred != bad_pred:
        if good_pred == lab_num:
            good_correct[i] = good_max - F.softmax(bad_model(im_torch)).data[0, good_pred]
        else:
            bad_correct[i] = bad_max - F.softmax(model(im_torch)).data[0, bad_pred]
            
    if i % 1000 == 0:
        print(i)

0


/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


1000
2000
3000
4000
5000
6000
7000
8000


Process Process-3:
Traceback (most recent call last):
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/accounts/projects/vision/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remainin

KeyboardInterrupt: 

In [ ]:
sorted_good = sorted(good_correct.items(), key=operator.itemgetter(1))[::-1]
sorted_bad = sorted(bad_correct.items(), key=operator.itemgetter(1))[::-1]
good_exs = set([a[0] for a in sorted_good[:6]])
bad_exs = set([a[0] for a in sorted_bad[:3]])

In [14]:
all_data = list(train_loader)

In [3]:
# Visualize good examples
for ind in good_exs:
    im = all_data[ind]
    im_torch, im_orig, lab_num =  Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0] #dset.get_im_and_label(1007)
    print(ind, lab_num, model.predict(im_torch), bad_model.predict(im_torch))
    mod = model
    lists = agglomerate.agglomerate(mod, pred_ims, percentile_include, 
                                    method, sweep_dim, layer, im_orig, 
                                    mod.predict(im_torch), use_abs, num_iters=num_iters, 
                                    im_torch=im_torch, model_type='mnist')

    # visualize
    plt.figure(figsize=(15, 9), facecolor='white')
    rows = 5
    viz.visualize_ims_list(lists['scores_list'], 
                           cmap_new='redwhiteblue',
                           title='Refined scores',
                           subplot_row=0, subplot_rows=rows, colorbar=True)
    viz.visualize_ims_list(lists['comps_list'],
                          title='blobs',
                          subplot_row=1, subplot_rows=rows, colorbar=False)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method,
                           subplot_row=2, subplot_rows=rows)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method, subplot_row=3, 
                            subplot_rows=rows, lab_num=lab_num, bar_graph=True)
    viz.visualize_arr_list(lists['comp_scores_raw_combined_list'], method, 
                           subplot_row=4, subplot_rows=rows)
    #plt.show()
    plt.savefig('vision_results/mnist/jamie/final/model2_' + str(ind) + '.png')

    #num_ims = lists['num_before_final']

    #viz.visualize_ims_list(lists['comps_list'], title='',  subplot_row=1, subplot_rows=rows, colorbar=False,
    #                       num_ims=lists['num_before_final'], im_orig=im_orig, plot_overlay=True,
    #                       mturk=True, comp_scores_raw=lists['comp_scores_raw_list'], lab_num_correct=lab_num, mnist=True, skip_first=True)
    #plt.show()
    
    mod = bad_model
    lists = agglomerate.agglomerate(mod, pred_ims, percentile_include, 
                                    method, sweep_dim, layer, im_orig, 
                                    mod.predict(im_torch), use_abs, num_iters=num_iters, 
                                    im_torch=im_torch, model_type='mnist')
    # visualize
    plt.figure(figsize=(15, 9), facecolor='white')
    rows = 5
    viz.visualize_ims_list(lists['scores_list'], 
                           cmap_new='redwhiteblue',
                           title='Refined scores',
                           subplot_row=0, subplot_rows=rows, colorbar=True)
    viz.visualize_ims_list(lists['comps_list'],
                          title='blobs',
                          subplot_row=1, subplot_rows=rows, colorbar=False)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method,
                           subplot_row=2, subplot_rows=rows)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method, subplot_row=3, 
                            subplot_rows=rows, lab_num=lab_num, bar_graph=True)
    viz.visualize_arr_list(lists['comp_scores_raw_combined_list'], method, 
                           subplot_row=4, subplot_rows=rows)
    #plt.show()
    plt.savefig('vision_results/mnist/jamie/final/model1_' + str(ind) + '.png')
    
    #num_ims = lists['num_before_final']

    #viz.visualize_ims_list(lists['comps_list'], title='',  subplot_row=1, subplot_rows=rows, colorbar=False,
    #                       num_ims=lists['num_before_final'], im_orig=im_orig, plot_overlay=True,
    #                       mturk=True, comp_scores_raw=lists['comp_scores_raw_list'], lab_num_correct=lab_num, mnist=True, skip_first=True)
    #plt.show()

NameError: name 'good_exs' is not defined

In [25]:
# Visualize bad examples
for ind in bad_exs:
    im = all_data[ind]
    im_torch, im_orig, lab_num =  Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0] #dset.get_im_and_label(1007)
    print(ind, lab_num, model.predict(im_torch), bad_model.predict(im_torch))
    mod = model
    lists = agglomerate.agglomerate(mod, pred_ims, percentile_include, 
                                    method, sweep_dim, layer, im_orig, 
                                    mod.predict(im_torch), use_abs, num_iters=num_iters, 
                                    im_torch=im_torch, model_type='mnist')

    # visualize
    plt.figure(figsize=(15, 9), facecolor='white')
    rows = 5
    viz.visualize_ims_list(lists['scores_list'], 
                           cmap_new='redwhiteblue',
                           title='Refined scores',
                           subplot_row=0, subplot_rows=rows, colorbar=True)
    viz.visualize_ims_list(lists['comps_list'],
                          title='blobs',
                          subplot_row=1, subplot_rows=rows, colorbar=False)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method,
                           subplot_row=2, subplot_rows=rows)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method, subplot_row=3, 
                            subplot_rows=rows, lab_num=lab_num, bar_graph=True)
    viz.visualize_arr_list(lists['comp_scores_raw_combined_list'], method, 
                           subplot_row=4, subplot_rows=rows)
    plt.savefig('vision_results/' + str(ind) + '_bad.png')

    mod = bad_model
    lists = agglomerate.agglomerate(mod, pred_ims, percentile_include, 
                                    method, sweep_dim, layer, im_orig, 
                                    mod.predict(im_torch), use_abs, num_iters=num_iters, 
                                    im_torch=im_torch, model_type='mnist')

    # visualize
    plt.figure(figsize=(15, 9), facecolor='white')
    rows = 5
    viz.visualize_ims_list(lists['scores_list'], 
                           cmap_new='redwhiteblue',
                           title='Refined scores',
                           subplot_row=0, subplot_rows=rows, colorbar=True)
    viz.visualize_ims_list(lists['comps_list'],
                          title='blobs',
                          subplot_row=1, subplot_rows=rows, colorbar=False)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method,
                           subplot_row=2, subplot_rows=rows)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method, subplot_row=3, 
                            subplot_rows=rows, lab_num=lab_num, bar_graph=True)
    viz.visualize_arr_list(lists['comp_scores_raw_combined_list'], method, 
                           subplot_row=4, subplot_rows=rows)
    plt.savefig('vision_results/' + str(ind) + '_bad.png')

SyntaxError: invalid syntax (<ipython-input-25-a0e240351820>, line 52)

In [16]:
print(good_exs)
print(bad_exs)

{51232, 43592, 22282, 4150, 11610, 15870}
{22779, 41396, 43574}


# generate final mturk viz examples

In [9]:
good_exs = [2116, 6782, 940, 462, 6749, 4702]
bad_exs = [9729, 1425, 1068] #[22779, 41396, 43574]
rows = 2
out_dir = 'vision_results/mnist'
for i, m in enumerate([model, bad_model]):
    model_name = ['model=good', 'model=bad'][i]
    for ex in good_exs + bad_exs:
        ex_fold = 'ex=good' if ex in good_exs else 'ex=bad'
        
        
        # get data[ex]
        im = all_data[ex]
        im_torch, im_orig, lab_num =  Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0] #dset.get_im_and_label(1007)


        # ACD
        fig_name = str(ex) + model_name + '_method=acd_' + ex_fold
        lists = agglomerate.agglomerate(m, pred_ims, percentile_include, 
                                    'cd', sweep_dim, layer, im_orig, 
                                    m.predict(im_torch), use_abs, num_iters=num_iters, 
                                    im_torch=im_torch, model_type='mnist')
        plt.figure(figsize=(12, 4), facecolor='white')
        ind, labs = viz.visualize_original_preds_mnist(im_orig, lab_num, 
                                             lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                             subplot_rows=rows, dset=dset, mturk=True)
        viz.visualize_ims_list(lists['comps_list'], title='',  subplot_row=1, subplot_rows=rows, colorbar=False,
                               num_ims=lists['num_before_final'], im_orig=im_orig, plot_overlay=True,
                               mturk=True, comp_scores_raw=lists['comp_scores_raw_list'], lab_num_correct=lab_num, mnist=True, skip_first=True)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

        # CD
        fig_name = str(ex) + model_name + '_method=cd_'+ ex_fold
        plt.figure(figsize=(16, 3), facecolor='white')
        ind, labs = viz.visualize_original_preds_mnist(im_orig, lab_num, 
                                             lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                             subplot_rows=1, dset=dset, mturk=True)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

        # integrated gradients
        fig_name = str(ex) + model_name + '_method=ig_'+ ex_fold
        scores_ig = score_funcs.ig_scores_cnn(model, im_torch)
        plt.figure(figsize=(16, 3), facecolor='white')
        ind, labs = viz.visualize_original_preds_mnist(im_orig, lab_num, 
                                             lists['comp_scores_raw_list'], scores_ig,
                                             subplot_rows=1, dset=dset, mturk=True, use_vmax=False)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight') 
        
        
        # break_down
        fig_name = str(ex) + model_name + '_method=break_down_'+ ex_fold
        tiles_break = tiling.gen_tiles(im_orig, fill=0, method='break_down', sweep_dim=1)
        preds_break = agglomerate.get_scores(model, method='break_down', ims=tiles_break, 
                                    im_torch=im_torch, pred_ims=pred_ims, layer='softmax')
        print(preds_break.shape)
        plt.figure(figsize=(16, 3), facecolor='white')
        ind, labs = viz.visualize_original_preds_mnist(im_orig, lab_num, 
                                             lists['comp_scores_raw_list'], preds_break,
                                             subplot_rows=1, dset=dset, mturk=True, use_vmax=False)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

SyntaxError: invalid syntax (<ipython-input-9-9ecb8ff2790d>, line 60)

# generate final supp examples

In [26]:
good_exs = [2116, 6782, 940, 462, 6749, 4702]
bad_exs = [9729, 1425, 1068]
rows = 3
out_dir = 'vision_results/mnist/supp'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
i = 0
m = model
model_name = ['model=good'][i]
for ex in good_exs + bad_exs:
    ex_fold = 'ex=good' if ex in good_exs else 'ex=bad'
    print(ex)

    # get data[ex]
    im = all_data[ex]
    im_torch, im_orig, lab_num =  Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0] #dset.get_im_and_label(1007)


    # ACD
    fig_name = str(ex) + '_supp_' + model_name + '_method=acd_' + ex_fold
    lists = agglomerate.agglomerate(m, pred_ims, percentile_include, 
                                'cd', sweep_dim, layer, im_orig, 
                                m.predict(im_torch), use_abs, num_iters=num_iters, 
                                im_torch=im_torch, model_type='mnist')

    # visualize
    plt.figure(figsize=(14, 6), facecolor='white', dpi=300)
    viz.visualize_ims_list(lists['scores_list'], 
                           cmap_new='redwhiteblue',
                           title='Refined scores',
                           subplot_row=0, subplot_rows=rows, colorbar=True)
    viz.visualize_ims_list(lists['comps_list'],
                          title='blobs',
                          subplot_row=1, subplot_rows=rows, colorbar=False)
    viz.visualize_dict_list(lists['comp_scores_raw_list'], method,
                           subplot_row=2, subplot_rows=rows)
    plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

SyntaxError: invalid syntax (<ipython-input-26-c7c29f3feff8>, line 37)